# ChatOutlines

该指南将帮助您开始使用 Outlines [chat models](/docs/concepts/chat_models/)。如需获取 ChatOutlines 所有功能和配置的详细文档，请访问 [API reference](https://python.langchain.com/api_reference/community/chat_models/langchain_community.chat_models.outlines.ChatOutlines.html)。

[Outlines](https://github.com/outlines-dev/outlines) 是一个用于受限语言生成的库。它允许您使用各种后端的大型语言模型（LLMs），同时对生成的输出应用约束。

## 概览
### 集成详情

| Class | Package | Local | Serializable | JS support | Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [ChatOutlines](https://python.langchain.com/api_reference/community/chat_models/langchain_community.chat_models.outlines.ChatOutlines.html) | [langchain-community](https://python.langchain.com/api_reference/community/index.html) | ✅ | ❌ | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-community?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-community?style=flat-square&label=%20) |

### 模型特性
| [Tool calling](/docs/how_to/tool_calling) | [Structured output](/docs/how_to/structured_output/) | JSON mode | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ✅ | ✅ | ❌ | ❌ | ✅ | ❌ | ❌ | ❌ | 

## 设置

要访问 Outlines 模型，您需要有互联网连接以下载模型权重从 huggingface。根据不同的后端，您需要安装所需的依赖项（请参阅 [Outlines docs](https://dottxt-ai.github.io/outlines/latest/installation/)）。

### 凭证

Outlines 没有内置的身份验证机制。

### 安装

LangChain Outlines 集成位于 `langchain-community` 包中，并且需要 `outlines` 库：

In [ ]:
%pip install -qU langchain-community outlines

## 实例化

现在我们可以实例化我们的模型对象并生成聊天补全：

In [ ]:
from langchain_community.chat_models.outlines import ChatOutlines

# For llamacpp backend
model = ChatOutlines(model="TheBloke/phi-2-GGUF/phi-2.Q4_K_M.gguf", backend="llamacpp")

# For vllm backend (not available on Mac)
model = ChatOutlines(model="meta-llama/Llama-3.2-1B", backend="vllm")

# For mlxlm backend (only available on Mac)
model = ChatOutlines(model="mistralai/Ministral-8B-Instruct-2410", backend="mlxlm")

# For huggingface transformers backend
model = ChatOutlines(model="microsoft/phi-2")  # defaults to transformers backend

## 调用

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="What will the capital of mars be called?")]
response = model.invoke(messages)

response.content

## 流式传输

ChatOutlines 支持向外流式传输 token：

In [ ]:
messages = [HumanMessage(content="Count to 10 in French:")]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

## 链式调用

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | model
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

## 受约束的生成

ChatOutlines 允许你对生成的输出应用各种约束：

### 正则表达式约束

In [ ]:
model.regex = r"((25[0-5]|2[0-4]\d|[01]?\d\d?)\.){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)"

response = model.invoke("What is the IP address of Google's DNS server?")

response.content

### 类型约束

In [ ]:
model.type_constraints = int
response = model.invoke("What is the answer to life, the universe, and everything?")

response.content

### Pydantic 和 JSON Schemas

In [ ]:
from pydantic import BaseModel


class Person(BaseModel):
    name: str


model.json_schema = Person
response = model.invoke("Who are the main contributors to LangChain?")
person = Person.model_validate_json(response.content)

person

### 上下文无关文法

In [ ]:
model.grammar = """
?start: expression
?expression: term (("+" | "-") term)*
?term: factor (("*" | "/") factor)*
?factor: NUMBER | "-" factor | "(" expression ")"
%import common.NUMBER
%import common.WS
%ignore WS
"""
response = model.invoke("Give me a complex arithmetic expression:")

response.content

## LangChain 的结构化输出

你也可以将 LangChain 的结构化输出与 ChatOutlines 结合使用：

In [ ]:
from pydantic import BaseModel


class AnswerWithJustification(BaseModel):
    answer: str
    justification: str


_model = model.with_structured_output(AnswerWithJustification)
result = _model.invoke("What weighs more, a pound of bricks or a pound of feathers?")

result

## API 参考

如需了解 ChatOutlines 所有功能和配置的详细文档，请前往 API 参考：https://python.langchain.com/api_reference/community/chat_models/langchain_community.chat_models.outlines.ChatOutlines.html

## Outlines 全套文档：

https://dottxt-ai.github.io/outlines/latest/